In [1]:
# Controle data .gouv
# 1. caracteristics
import pandas as pd
import numpy as np

# Chemins d'accès aux données
repData = '../../data/raw/'
repDataConcat = '../../data/concat/'

# Liste des fichiers vehicules disponibles
!ls {repData}ve* -lrt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

ls: -lrt: No such file or directory
../../data/raw/vehicules-2017.csv ../../data/raw/vehicules_2008.csv
../../data/raw/vehicules-2018.csv ../../data/raw/vehicules_2009.csv
../../data/raw/vehicules-2019.csv ../../data/raw/vehicules_2010.csv
../../data/raw/vehicules-2020.csv ../../data/raw/vehicules_2011.csv
../../data/raw/vehicules-2021.csv ../../data/raw/vehicules_2012.csv
../../data/raw/vehicules-2022.csv ../../data/raw/vehicules_2013.csv
../../data/raw/vehicules_2005.csv ../../data/raw/vehicules_2014.csv
../../data/raw/vehicules_2006.csv ../../data/raw/vehicules_2015.csv
../../data/raw/vehicules_2007.csv ../../data/raw/vehicules_2016.csv


#### Consolidation de 'vehicules.csv' - modus operandi
0. Chgt de tous les fichiers 
      - nom des fichiers de 2005 à 2016 : '_'
      - nom des fichiers de 2017 à 2022 : remplacement de '_' par '-'
      - séparateur ',' sauf 2009='\t' et 2019 à 2022=';'

1. Variable 'Num_Acc' renommée en 'id_accident' pour être iso entre les 4 types de fichiers et forcé en 'int64'
2. Variable 'id_vehicule' : apparition en 2019 en complément du 'Num_veh' (disp également dans le fichier usager)
      - Remplacement de '\xa0' par '' --> transtypage de la variable en int64 
      - Création d'une colonne 'id_vehicule' dans tous les fichiers antérieurs à 2019 : initialisée à 0 afin de permettre des jointures avec usager par la suite
 
3. Variable 'senc' : Remplacement dans tous les fichiers :
      - Valeur NaN  --> 0=inconnu (apparue en 2019)
      - Valeur '-1' --> 0=inconnu (apparue en 2019)
      - forcer le type en int64

4. Variable 'catv' : remplacer les valeurs plus utilisées depuis 2006 par leur(s) valeur(s) cible(s)
     
      - Si plusieurs valeurs possible comme 05 (motocyclette) --> 31='' ou 33='', on tiendra compte de la distribution de ces valeurs pour remplace aléatoirement les 5 par l'une des deux valeurs remplaçante

      - Bien que ces codes aient diparus en 2006, nous en retrouvons jusqu'en 2008 : reprise des fichiers jusqu'à 2008 inclu
      - Règle de remplacement implémentée :
          - 04 --> 32 ou 34   (transco à vérifier !!!)
          - 05 --> 31 ou 33
          - 06 --> 33
          - 08 --> 07
          - 09 --> 07
          - 11 --> 10
          - 12 --> 10
          - 18 --> 37 ou 38
          - 19 --> 40  
5. Variable 'obs' :
       - Forcer les NaN --> -1 pour être cohérent avec les fichiers à partir de 2019 qui ajoute cette valeur -1
       - La valeur 17 ajoutée en 2019 ne devrait pas poser de problème vu le nombre d'occurrence (marginale)
       - Typer la Variable en 'int64' (catégorielle)
6. Variable 'obsm' :
       - Forcer les NaN --> -1 pour être cohérent avec les fichiers à partir de 2019 qui ajoute cette valeur -1
       - Typer la Variable en 'int64' (catégorielle)
7. Variable 'choc' :
       - Forcer les NaN --> -1 pour être cohérent avec les fichiers à partir de 2019 qui ajoute cette valeur -1
       - Typer la Variable en 'int64' (catégorielle)
8. Variable 'manv' :
       - Forcer les NaN --> -1 pour être cohérent avec les fichiers à partir de 2019 qui ajoute cette valeur -1
       - Typer la Variable en 'int64' (catégorielle)
       - Deux nouvelles valeurs 26 et 26 apparaissent en 2019 (assez marginal)
9. Variable 'motor' : apparait à partir de 2019
       - Création d'une colonne 'motor' initialisée à NaN pour les fichiers antérieurs : l'info ne pourra pas être exploitée à ces dates
       - Toutes les données sont renseignées, les NaN étant positionnées à -1.
10. Variable occutc : Détection d'un changement de règle de gestion : cette variable ne concerne que les véhicules de transport en commum mais jusqu'à 2018, nous retrouvions 0 dans cette variable alors qu'à partir de 2019, elle n'est plus renseignée.
    Un contrôle permet également de vérifier que la valeur est bien soit à 0 ou à NaN pour les véhicule "non transport en commun afin de passer à 0 sa valeur (erreur de saisie).
11. Ordonner les colonnes de chaque df dans le même ordre
12. Concaténation en un seul fichier 
13. Variable 'catv' année 2005, remplacement des valeurs qui n'ont pu l'être précédemment
14. Libération de mémoire      
15. Controles
16. Suppression des colonnes inutiles
17. Sauvegarde du df consolidé sous un fichier caracteristiques.csv

In [2]:
# 0. chargement et merge de tous les fichiers de 2005 à 2016 --> carac
df_2005 = pd.read_csv(repData + "vehicules_2005.csv", sep = ',')
df_2006 = pd.read_csv(repData + "vehicules_2006.csv", sep = ',')
df_2007 = pd.read_csv(repData + "vehicules_2007.csv", sep = ',')
df_2008 = pd.read_csv(repData + "vehicules_2008.csv", sep = ',')
df_2009 = pd.read_csv(repData + "vehicules_2009.csv", sep = ',')
df_2010 = pd.read_csv(repData + "vehicules_2010.csv", sep = ',')
df_2011 = pd.read_csv(repData + "vehicules_2011.csv", sep = ',')
df_2012 = pd.read_csv(repData + "vehicules_2012.csv", sep = ',')
df_2013 = pd.read_csv(repData + "vehicules_2013.csv", sep = ',')
df_2014 = pd.read_csv(repData + "vehicules_2014.csv", sep = ',')
df_2015 = pd.read_csv(repData + "vehicules_2015.csv", sep = ',')
df_2016 = pd.read_csv(repData + "vehicules_2016.csv", sep = ',')
df_2017 = pd.read_csv(repData + "vehicules-2017.csv", sep = ',')
df_2018 = pd.read_csv(repData + "vehicules-2018.csv", sep = ',')
df_2019 = pd.read_csv(repData + "vehicules-2019.csv", sep = ';')
df_2020 = pd.read_csv(repData + "vehicules-2020.csv", sep = ';')
df_2021 = pd.read_csv(repData + "vehicules-2021.csv", sep = ';')
df_2022 = pd.read_csv(repData + "vehicules-2022.csv", sep = ';')



In [3]:
# 1. Renommer la colonne  'Num_Acc' (et 'Accident_Id' en 2022) en 'id_accident' qui servira de clé entre les 4 types de fichiers
rename_columns = {'Num_Acc' : 'id_accident'}
df_2005 = df_2005.rename(columns = rename_columns)
df_2006 = df_2006.rename(columns = rename_columns)
df_2007 = df_2007.rename(columns = rename_columns)
df_2008 = df_2008.rename(columns = rename_columns)
df_2009 = df_2009.rename(columns = rename_columns)
df_2010 = df_2010.rename(columns = rename_columns)
df_2011 = df_2011.rename(columns = rename_columns)
df_2012 = df_2012.rename(columns = rename_columns)
df_2013 = df_2013.rename(columns = rename_columns)
df_2014 = df_2014.rename(columns = rename_columns)
df_2015 = df_2015.rename(columns = rename_columns)
df_2016 = df_2016.rename(columns = rename_columns)
df_2017 = df_2017.rename(columns = rename_columns)
df_2018 = df_2018.rename(columns = rename_columns)
df_2019 = df_2019.rename(columns = rename_columns)
df_2020 = df_2020.rename(columns = rename_columns)
df_2021 = df_2021.rename(columns = rename_columns)
df_2022 = df_2022.rename(columns = rename_columns)



In [4]:
# 2.1 Variable 'id_vehicule' ajouté dans tous les fichiers antérieurs à 2019 et initialisé à '0' afin de permettre des jointures avec usagers
df_2005['id_vehicule'] = 0
df_2006['id_vehicule'] = 0
df_2007['id_vehicule'] = 0
df_2008['id_vehicule'] = 0
df_2009['id_vehicule'] = 0
df_2010['id_vehicule'] = 0
df_2011['id_vehicule'] = 0
df_2012['id_vehicule'] = 0
df_2013['id_vehicule'] = 0
df_2014['id_vehicule'] = 0
df_2015['id_vehicule'] = 0
df_2016['id_vehicule'] = 0
df_2017['id_vehicule'] = 0
df_2018['id_vehicule'] = 0

# 2.2 Variable 'id_vehicule' : retirer le séparateur de millier ' ' --> transtyper la variable en 'int64'
df_2005['id_vehicule'] = df_2005['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2006['id_vehicule'] = df_2006['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2007['id_vehicule'] = df_2007['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2008['id_vehicule'] = df_2008['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2009['id_vehicule'] = df_2009['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2010['id_vehicule'] = df_2010['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2011['id_vehicule'] = df_2011['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2012['id_vehicule'] = df_2012['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2013['id_vehicule'] = df_2013['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2014['id_vehicule'] = df_2014['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2015['id_vehicule'] = df_2015['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2016['id_vehicule'] = df_2016['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2017['id_vehicule'] = df_2017['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2018['id_vehicule'] = df_2018['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2019['id_vehicule'] = df_2019['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2020['id_vehicule'] = df_2020['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2021['id_vehicule'] = df_2021['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2022['id_vehicule'] = df_2022['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)


In [5]:
# 3. Variable 'senc' : passer les NaN et les '-1' --> 0 : inconnu et forcer le type en 'int64'
df_2005["senc"] = df_2005["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2006["senc"] = df_2006["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2007["senc"] = df_2007["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2008["senc"] = df_2008["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2009["senc"] = df_2009["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2010["senc"] = df_2010["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2011["senc"] = df_2011["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2012["senc"] = df_2012["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2013["senc"] = df_2013["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2014["senc"] = df_2014["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2015["senc"] = df_2015["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2016["senc"] = df_2016["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2017["senc"] = df_2017["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2018["senc"] = df_2018["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2019["senc"] = df_2019["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2020["senc"] = df_2020["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2021["senc"] = df_2021["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)
df_2022["senc"] = df_2022["senc"].apply(lambda valeur : 0 if (valeur == -1) or (valeur == np.nan) else valeur)


In [6]:
# 4.0 Variable 'catv', Définition fonction de remplacement fonction de remplacement à privilégier si 2 valeurs cibles sont possible
from typing import List
def remplacement_catv (df:pd.DataFrame, colonne:str, valeurs: List[str]) -> np.array:
    """ Permet de remplacer dans df['colonne'] la première valeur définie dans la liste valeurs par les autres valeurs de la liste selon leur répartition
        dans df['colonne'].

        Args :
           - df : DataFrame à traiter : il ne sera pas modifié (logique pd)
           - colonne (string) : nom  de la colonne à traiter
           - valeurs (List[str]) : contiendra en position 0 la valeur à modifier, dans le reste, les valeurs à prendre en compte pour
                                   réaliser cette mise à jour
        
        Return : 
           - vecteur df[colonne] avec les valeurs valeurs[0] remplacées par valeurs[n] n>0 en gardant la distribution actuelle de ces
             valeurs dans df[colonne]
             
    """    
    try :
        # Ne pas modifier le df initial (logique pandas)
        df_copy = df.copy()
        df_copy[colonne] = df_copy[colonne].astype(str)

        # Si valeurs est un tableau de 2 valeurs alors demande de remplacement de X --> Y : l'opération sera possible même si aucune occurrence d'Y
        if len(valeurs) == 2 :
            df_copy[colonne] = df_copy[colonne].apply(lambda valeur : valeurs[1] if valeur == valeurs[0] else valeur)

        # Sinon (valeurs.shape[1]>2
        else:
            # Vérifie qu'il y a bien des données à modifier
            nb_valeurs_a_modifier = df_copy[(df_copy[colonne] == valeurs[0])][colonne].count()
            if nb_valeurs_a_modifier == 0 :
                name=df.id_accident[0].astype(str)[:4]
                #print (f">>> Aucune valeur à corriger dans le df_{name}, pour la valeur={valeurs[0: 1]}")
                return df[colonne]
                
            # Filtrer les valeurs autres que celles contenues dans valeur
            valeurs_filtréesRef = df_copy[colonne].loc[df_copy[colonne].isin(valeurs[1:])]

            if len(valeurs_filtréesRef) == 0 :
                # Aucune valeur de remplacement trouvée --> action impossible
                name=df.id_accident[0].astype(str)[:4]
                print (f">>> Aucune valeur de remplacement trouvée dans le df_{name}, valeur={valeurs[0]} --> {valeurs[1:]}, nb enregistrements={nb_valeurs_a_modifier}. Opération impossible.")
                return df[colonne]
               
            # Calculer la distribution actuelle des valeurs
            distribution_actuelle = valeurs_filtréesRef.value_counts(normalize=True)
    
            # Remplacer la valeurs[0] par les nouvelles valeurs[n] n>0 avec la même distribution
            nouvelles_valeurs = np.random.choice(distribution_actuelle.index, size=len(df_copy), p=distribution_actuelle.values)
            df_copy[colonne] = np.where(df_copy[colonne] == valeurs[0], nouvelles_valeurs, df_copy[colonne])

        return df_copy[colonne].astype(df_copy[colonne].dtype)

    except Exception as err :
        # Problème rencontré
        print (f'Problème rencontré : {err}. Opération annulé.')

        return None


In [7]:
# 4. Variable catv : remplacement des valeurs 
df_2005['catv'] = remplacement_catv(df_2005, 'catv', ['4', '32', '34'])
df_2005['catv'] = remplacement_catv(df_2005, 'catv', ['5', '31', '33'])
df_2005['catv'] = remplacement_catv(df_2005, 'catv', ['6', '33'])
df_2005['catv'] = remplacement_catv(df_2005, 'catv', ['8', '7'])
df_2005['catv'] = remplacement_catv(df_2005, 'catv', ['9', '7'])
df_2005['catv'] = remplacement_catv(df_2005, 'catv', ['11', '10'])
df_2005['catv'] = remplacement_catv(df_2005, 'catv', ['12', '10'])
df_2005['catv'] = remplacement_catv(df_2005, 'catv', ['18', '37', '38'])
df_2005['catv'] = remplacement_catv(df_2005, 'catv', ['19', '40'])

df_2006['catv'] = remplacement_catv(df_2006, 'catv', ['4', '32', '34'])
df_2006['catv'] = remplacement_catv(df_2006, 'catv', ['5', '31', '33'])
df_2006['catv'] = remplacement_catv(df_2006, 'catv', ['6', '33'])
df_2006['catv'] = remplacement_catv(df_2006, 'catv', ['8', '7'])
df_2006['catv'] = remplacement_catv(df_2006, 'catv', ['9', '7'])
df_2006['catv'] = remplacement_catv(df_2006, 'catv', ['11', '10'])
df_2006['catv'] = remplacement_catv(df_2006, 'catv', ['12', '10'])
df_2006['catv'] = remplacement_catv(df_2006, 'catv', ['18', '37', '38'])
df_2006['catv'] = remplacement_catv(df_2006, 'catv', ['19', '40'])

df_2007['catv'] = remplacement_catv(df_2007, 'catv', ['4', '32', '34'])
df_2007['catv'] = remplacement_catv(df_2007, 'catv', ['5', '31', '33'])
df_2007['catv'] = remplacement_catv(df_2007, 'catv', ['6', '33'])
df_2007['catv'] = remplacement_catv(df_2007, 'catv', ['8', '7'])
df_2007['catv'] = remplacement_catv(df_2007, 'catv', ['9', '7'])
df_2007['catv'] = remplacement_catv(df_2007, 'catv', ['11', '10'])
df_2007['catv'] = remplacement_catv(df_2007, 'catv', ['12', '10'])
df_2007['catv'] = remplacement_catv(df_2007, 'catv', ['18', '37', '38'])
df_2007['catv'] = remplacement_catv(df_2007, 'catv', ['19', '40'])

df_2008['catv'] = remplacement_catv(df_2008, 'catv', ['4', '32', '34'])
df_2008['catv'] = remplacement_catv(df_2008, 'catv', ['5', '31', '33'])
df_2008['catv'] = remplacement_catv(df_2008, 'catv', ['6', '33'])
df_2008['catv'] = remplacement_catv(df_2008, 'catv', ['8', '7'])
df_2008['catv'] = remplacement_catv(df_2008, 'catv', ['9', '7'])
df_2008['catv'] = remplacement_catv(df_2008, 'catv', ['11', '10'])
df_2008['catv'] = remplacement_catv(df_2008, 'catv', ['12', '10'])
df_2008['catv'] = remplacement_catv(df_2008, 'catv', ['18', '37', '38'])
df_2008['catv'] = remplacement_catv(df_2008, 'catv', ['19', '40'])

# Les valeurs que l'on n'arrive pas à transcoder devront l'être une fois la concaténation réalisée.
# On ne tiendra plus compte de la distribution à l'année mais celle de l'ensemble de la période

>>> Aucune valeur de remplacement trouvée dans le df_2005, valeur=4 --> ['32', '34'], nb enregistrements=4338. Opération impossible.
>>> Aucune valeur de remplacement trouvée dans le df_2005, valeur=5 --> ['31', '33'], nb enregistrements=14859. Opération impossible.
>>> Aucune valeur de remplacement trouvée dans le df_2005, valeur=18 --> ['37', '38'], nb enregistrements=1360. Opération impossible.


In [8]:
# 5. Variable 'obs' : les valeurs NaN --> -1 et typage en 'int64'
df_2005['obs'] = df_2005['obs'].fillna(-1).astype(int)
df_2006['obs'] = df_2006['obs'].fillna(-1).astype(int)
df_2007['obs'] = df_2007['obs'].fillna(-1).astype(int)
df_2008['obs'] = df_2008['obs'].fillna(-1).astype(int)
df_2009['obs'] = df_2009['obs'].fillna(-1).astype(int)
df_2010['obs'] = df_2010['obs'].fillna(-1).astype(int)
df_2011['obs'] = df_2011['obs'].fillna(-1).astype(int)
df_2012['obs'] = df_2012['obs'].fillna(-1).astype(int)
df_2013['obs'] = df_2013['obs'].fillna(-1).astype(int)
df_2014['obs'] = df_2014['obs'].fillna(-1).astype(int)
df_2015['obs'] = df_2015['obs'].fillna(-1).astype(int)
df_2016['obs'] = df_2016['obs'].fillna(-1).astype(int)
df_2017['obs'] = df_2017['obs'].fillna(-1).astype(int)
df_2018['obs'] = df_2018['obs'].fillna(-1).astype(int)
df_2019['obs'] = df_2019['obs'].fillna(-1).astype(int)
df_2020['obs'] = df_2020['obs'].fillna(-1).astype(int)
df_2021['obs'] = df_2021['obs'].fillna(-1).astype(int)
df_2022['obs'] = df_2022['obs'].fillna(-1).astype(int)


In [9]:
# 6. Variable 'obsm' : les valeurs NaN --> -1 et typage en 'int64'
df_2005['obsm'] = df_2005['obsm'].fillna(-1).astype(int)
df_2006['obsm'] = df_2006['obsm'].fillna(-1).astype(int)
df_2007['obsm'] = df_2007['obsm'].fillna(-1).astype(int)
df_2008['obsm'] = df_2008['obsm'].fillna(-1).astype(int)
df_2009['obsm'] = df_2009['obsm'].fillna(-1).astype(int)
df_2010['obsm'] = df_2010['obsm'].fillna(-1).astype(int)
df_2011['obsm'] = df_2011['obsm'].fillna(-1).astype(int)
df_2012['obsm'] = df_2012['obsm'].fillna(-1).astype(int)
df_2013['obsm'] = df_2013['obsm'].fillna(-1).astype(int)
df_2014['obsm'] = df_2014['obsm'].fillna(-1).astype(int)
df_2015['obsm'] = df_2015['obsm'].fillna(-1).astype(int)
df_2016['obsm'] = df_2016['obsm'].fillna(-1).astype(int)
df_2017['obsm'] = df_2017['obsm'].fillna(-1).astype(int)
df_2018['obsm'] = df_2018['obsm'].fillna(-1).astype(int)
df_2019['obsm'] = df_2019['obsm'].fillna(-1).astype(int)
df_2020['obsm'] = df_2020['obsm'].fillna(-1).astype(int)
df_2021['obsm'] = df_2021['obsm'].fillna(-1).astype(int)
df_2022['obsm'] = df_2022['obsm'].fillna(-1).astype(int)


In [10]:
# 7. Variable 'choc' : les valeurs NaN --> -1 et typage en 'int64'
df_2005['choc'] = df_2005['choc'].fillna(-1).astype(int)
df_2006['choc'] = df_2006['choc'].fillna(-1).astype(int)
df_2007['choc'] = df_2007['choc'].fillna(-1).astype(int)
df_2008['choc'] = df_2008['choc'].fillna(-1).astype(int)
df_2009['choc'] = df_2009['choc'].fillna(-1).astype(int)
df_2010['choc'] = df_2010['choc'].fillna(-1).astype(int)
df_2011['choc'] = df_2011['choc'].fillna(-1).astype(int)
df_2012['choc'] = df_2012['choc'].fillna(-1).astype(int)
df_2013['choc'] = df_2013['choc'].fillna(-1).astype(int)
df_2014['choc'] = df_2014['choc'].fillna(-1).astype(int)
df_2015['choc'] = df_2015['choc'].fillna(-1).astype(int)
df_2016['choc'] = df_2016['choc'].fillna(-1).astype(int)
df_2017['choc'] = df_2017['choc'].fillna(-1).astype(int)
df_2018['choc'] = df_2018['choc'].fillna(-1).astype(int)
df_2019['choc'] = df_2019['choc'].fillna(-1).astype(int)
df_2020['choc'] = df_2020['choc'].fillna(-1).astype(int)
df_2021['choc'] = df_2021['choc'].fillna(-1).astype(int)
df_2022['choc'] = df_2022['choc'].fillna(-1).astype(int)


In [11]:
# 8. Variable 'manv' : les valeurs NaN --> -1 et typage en 'int64'
df_2005['manv'] = df_2005['manv'].fillna(-1).astype(int)
df_2006['manv'] = df_2006['manv'].fillna(-1).astype(int)
df_2007['manv'] = df_2007['manv'].fillna(-1).astype(int)
df_2008['manv'] = df_2008['manv'].fillna(-1).astype(int)
df_2009['manv'] = df_2009['manv'].fillna(-1).astype(int)
df_2010['manv'] = df_2010['manv'].fillna(-1).astype(int)
df_2011['manv'] = df_2011['manv'].fillna(-1).astype(int)
df_2012['manv'] = df_2012['manv'].fillna(-1).astype(int)
df_2013['manv'] = df_2013['manv'].fillna(-1).astype(int)
df_2014['manv'] = df_2014['manv'].fillna(-1).astype(int)
df_2015['manv'] = df_2015['manv'].fillna(-1).astype(int)
df_2016['manv'] = df_2016['manv'].fillna(-1).astype(int)
df_2017['manv'] = df_2017['manv'].fillna(-1).astype(int)
df_2018['manv'] = df_2018['manv'].fillna(-1).astype(int)
df_2019['manv'] = df_2019['manv'].fillna(-1).astype(int)
df_2020['manv'] = df_2020['manv'].fillna(-1).astype(int)
df_2021['manv'] = df_2021['manv'].fillna(-1).astype(int)
df_2022['manv'] = df_2022['manv'].fillna(-1).astype(int)


In [12]:
# 9. Variable 'motor' - Création de la colonne sur les df antérieurs à 2019

df_2005['motor'] = np.nan
df_2006['motor'] = np.nan
df_2007['motor'] = np.nan
df_2008['motor'] = np.nan
df_2009['motor'] = np.nan
df_2010['motor'] = np.nan
df_2011['motor'] = np.nan
df_2012['motor'] = np.nan
df_2013['motor'] = np.nan
df_2014['motor'] = np.nan
df_2015['motor'] = np.nan
df_2016['motor'] = np.nan
df_2017['motor'] = np.nan
df_2018['motor'] = np.nan


In [13]:
df_2020[(df_2020.id_accident==202000000001)]

,id_accident,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc
0,202000000001,154742274,B01,1,7,0,2,2,15,1,NaN
1,202000000001,154742275,A01,1,33,0,2,1,2,1,NaN


In [14]:
# 10.1 Variable 'occutc' : Controle de cohérence de occutc qui doit être soit à 0, soit à NaN pour les catégories de véhicule non transport en commun
liste_id_accident = []
liste_df = [df_2005, df_2006, df_2007, df_2008]        # df identifié avec des incohérence de données
for df in liste_df :
    for index, row in df[(~df['catv'].isin([18, 19, 37, 38, 39, 40])) & (df['occutc'].fillna(0) != 0)].iterrows():
       #print(row)
       liste_id_accident.append(row['id_accident'])

# Génération d'une liste d'id_accident incohérent --> pour controle dans fichier agrégé afin de retrouver le nombre de passager pour chacun de ces véhicules 
# et ainsi confirmer que l'anomalie ne concerne pas la catégorisation du véhicule mais bien le champs occutc
liste_id_accident

# Correction confirmée --> on force à 0 occutc pour tous les accidents dont l'id est présent dans liste_idiaccident
for df in liste_df :
    for index, row in df[(~df['catv'].isin([18, 19, 37, 38, 39, 40])) & (df['occutc'].fillna(0) != 0)].iterrows():
        df.loc[index, 'occutc'] = 0

In [15]:
# 10.2 Variable 'occutc' - Forcer à 0 les valeurs de occutc à NaN dans tous les df
df_2005['occutc'] = df_2005['occutc'].fillna(0).astype(int)
df_2006['occutc'] = df_2006['occutc'].fillna(0).astype(int)
df_2007['occutc'] = df_2007['occutc'].fillna(0).astype(int)
df_2008['occutc'] = df_2008['occutc'].fillna(0).astype(int)
df_2009['occutc'] = df_2009['occutc'].fillna(0).astype(int)
df_2010['occutc'] = df_2010['occutc'].fillna(0).astype(int)
df_2011['occutc'] = df_2011['occutc'].fillna(0).astype(int)
df_2012['occutc'] = df_2012['occutc'].fillna(0).astype(int)
df_2013['occutc'] = df_2013['occutc'].fillna(0).astype(int)
df_2014['occutc'] = df_2014['occutc'].fillna(0).astype(int)
df_2015['occutc'] = df_2015['occutc'].fillna(0).astype(int)
df_2016['occutc'] = df_2016['occutc'].fillna(0).astype(int)
df_2017['occutc'] = df_2017['occutc'].fillna(0).astype(int)
df_2018['occutc'] = df_2018['occutc'].fillna(0).astype(int)
df_2019['occutc'] = df_2019['occutc'].fillna(0).astype(int)
df_2020['occutc'] = df_2020['occutc'].fillna(0).astype(int)
df_2021['occutc'] = df_2021['occutc'].fillna(0).astype(int)
df_2022['occutc'] = df_2022['occutc'].fillna(0).astype(int)


In [16]:
# 10. Réindexer les colonnes de chaque df dans le même ordre
ordre_colonnes = ['id_accident', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm', 'choc', 'manv', 'motor', 'occutc']
df_2005 = df_2005.reindex(columns=ordre_colonnes)
df_2006 = df_2006.reindex(columns=ordre_colonnes)
df_2007 = df_2007.reindex(columns=ordre_colonnes)
df_2008 = df_2008.reindex(columns=ordre_colonnes)
df_2009 = df_2009.reindex(columns=ordre_colonnes)
df_2010 = df_2010.reindex(columns=ordre_colonnes)
df_2011 = df_2011.reindex(columns=ordre_colonnes)
df_2012 = df_2012.reindex(columns=ordre_colonnes)
df_2013 = df_2013.reindex(columns=ordre_colonnes)
df_2014 = df_2014.reindex(columns=ordre_colonnes)
df_2015 = df_2015.reindex(columns=ordre_colonnes)
df_2016 = df_2016.reindex(columns=ordre_colonnes)
df_2017 = df_2017.reindex(columns=ordre_colonnes)
df_2018 = df_2018.reindex(columns=ordre_colonnes)
df_2019 = df_2019.reindex(columns=ordre_colonnes)
df_2020 = df_2020.reindex(columns=ordre_colonnes)
df_2021 = df_2021.reindex(columns=ordre_colonnes)
df_2022 = df_2022.reindex(columns=ordre_colonnes)


In [17]:
# 11. Concaténer les df en un seul
vehi = df_2005.copy()
vehi = pd.concat ([vehi, df_2006], axis = 0)
vehi = pd.concat ([vehi, df_2007], axis = 0)
vehi = pd.concat ([vehi, df_2008], axis = 0)
vehi = pd.concat ([vehi, df_2009], axis = 0)
vehi = pd.concat ([vehi, df_2010], axis = 0)
vehi = pd.concat ([vehi, df_2011], axis = 0)
vehi = pd.concat ([vehi, df_2012], axis = 0)
vehi = pd.concat ([vehi, df_2013], axis = 0)
vehi = pd.concat ([vehi, df_2014], axis = 0)
vehi = pd.concat ([vehi, df_2015], axis = 0)
vehi = pd.concat ([vehi, df_2016], axis = 0)
vehi = pd.concat ([vehi, df_2017], axis = 0)
vehi = pd.concat ([vehi, df_2018], axis = 0)
vehi = pd.concat ([vehi, df_2019], axis = 0)
vehi = pd.concat ([vehi, df_2020], axis = 0)
vehi = pd.concat ([vehi, df_2021], axis = 0)
vehi = pd.concat ([vehi, df_2022], axis = 0)


In [18]:
# 12. Variable 'catv' - Remplacmeent des 2 velaurs qui n'ont pu m'être en 3 
vehi['catv'] = remplacement_catv(vehi, 'catv', ['4', '32', '34'])
vehi['catv'] = remplacement_catv(vehi, 'catv', ['5', '31', '33'])

In [19]:
# 13. Suppression des objets inutiles --> libération de mémoire
del df_2005
del df_2006
del df_2007
del df_2008
del df_2009
del df_2010
del df_2011
del df_2012
del df_2013
del df_2014
del df_2015
del df_2016
del df_2017
del df_2018
del df_2019
del df_2020
del df_2021
del df_2022

In [20]:
# 14. Controles et ajustements
vehi.occutc.count()
#vehi.occutc.isna().sum()
# 14.1 Ctrl des valeurs de 'catv'
#horsNorme_catv = vehi[(vehi['catv'].isin([1, 2, 3, 7, 10, 13, 14, 15, 16, 17, 20, 21, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 99]), 'catv']
#horsNorme_catv = vehi['catv'].isin([-1, 0, 1, 2, 3, 7, 10, 13, 14, 15, 16, 17, 20, 21, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 50, 60, 80, 99]).all()
#print ("horsNorme_catv=", horsNorme_catv)

#horsNorme_obs = vehi[(vehi['obs']>= -1) & (vehi['obs'] <= 17), 'obs']
#print ("horsNorme_obs=", horsNorme_catv)

#vehi['senc'].value_counts().sort_index()
#vehi.head(50)
#vehi.catv.value_counts()
#vehi.describe()
#vehi.info()

2009395

In [21]:
# 15. Sauvegarde du fichier en csv (sans index) 
vehi.to_csv(repDataConcat +"vehicules.csv", index=False, encoding='utf-8')

# 16. Sauvegarde du fichier en csv (sans index) de 2022 pour préannalyse statistique des données
#df_2022.to_csv(repDataConcat +"vehicules_2022.csv", index=False, encoding='utf-8')